In [172]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding, RepeatVector
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import string
import re


In [485]:
training_df = pd.read_csv('./data.txt', delimiter='\n', header=None)
labels = pd.read_csv('./labels.txt', delimiter='\n', header=None)


FileNotFoundError: [Errno 2] No such file or directory: './data.txt'

In [486]:
def tokenize(line):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(line)
    return tokenizer

In [487]:
training_tok = tokenize(training_df[0])
training_size = len(training_tok.word_index) + 1
training_length = 16
print(training_size)

533


In [478]:
label_tok = tokenize(labels[0])
label_size = len(label_tok.word_index) + 1
label_length = 6
print(label_size)

217


In [479]:
def encode_sequences(tokenizer, length, lines):
    seq = tokenizer.texts_to_sequences(lines)
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

In [480]:
X_train = training_df
Y_train = labels

In [488]:
X_train, Y_train

(                                                     0
 0    8 ounces extra firm tofu, cut into 1/4-inch cubes
 1                                 8 ounces small pasta
 2                            6 tablespoons ponzu sauce
 3                              3 garlic cloves, minced
 4         scant 1/2 teaspoon chile flakes, or to taste
 ..                                                 ...
 398  1 tablespoon extra-virgin olive oil (or clarif...
 399                                 2 cups masa harina
 400                                   1 1/2 cups water
 401              2 cups quinoa well-rinsed and drained
 402                                   3 1/3 cups water
 
 [403 rows x 1 columns],
                         0
 0               8 oz tofu
 1              8 oz pasta
 2      6 tbsp ponzu sauce
 3         3 garlic cloves
 4    0.5 tsp chilli flake
 ..                    ...
 398      1 tbsp olive oil
 399     2 cup masa harina
 400         1.5 cup water
 401          2 cup quinoa
 402

In [451]:
X_test = pd.read_csv('./data_test.txt', delimiter='\n')

In [452]:
X_test

,string
0,1/2 tsp fine grain sea salt
1,3 tablespoons olive oil
2,1/4 cup shoyu sauce (or soy sauce)
3,2 tablespoons balsamic vinegar
4,2 tablespoons brown sugar (or maple syrup)


In [491]:
X_train = encode_sequences(training_tok, training_length, X_train[0])
Y_train = encode_sequences(label_tok, label_length, Y_train[0])
#X_test = encode_sequences(training_tok, training_length, X_test['string'])
    
# Y_test = encode_sequences(label_tok, label_length, Y_test['string'])

In [492]:
def define_model(in_vocab, out_vocab, in_timesteps, out_timesteps, units):
    model = Sequential()
    model.add(Embedding(in_vocab, units, input_length=in_timesteps, mask_zero=True))
    model.add(LSTM(units))
    model.add(RepeatVector(out_timesteps))
    model.add(LSTM(units, return_sequences=True))
    model.add(Dense(out_vocab, activation='softmax'))
    return model

In [493]:
model = define_model(training_size, label_size, training_length, label_length, 512)
rms = optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy')

In [494]:
history = model.fit(X_train, Y_train.reshape(Y_train.shape[0], Y_train.shape[1], 1), 
epochs=10, batch_size=100, validation_split=0.1)

Epoch 1/10
4/4 [==============================] - 12s 1s/step - loss: 5.0191 - val_loss: 3.2136
Epoch 2/10
4/4 [==============================] - 2s 505ms/step - loss: 3.2459 - val_loss: 2.8959
Epoch 3/10
4/4 [==============================] - 2s 516ms/step - loss: 2.8903 - val_loss: 2.8420
Epoch 4/10
4/4 [==============================] - 2s 508ms/step - loss: 2.9007 - val_loss: 2.7691
Epoch 5/10
4/4 [==============================] - 2s 483ms/step - loss: 2.7013 - val_loss: 2.7065
Epoch 6/10
4/4 [==============================] - 2s 559ms/step - loss: 2.6915 - val_loss: 2.6021
Epoch 7/10
4/4 [==============================] - 2s 595ms/step - loss: 2.4844 - val_loss: 2.5987
Epoch 8/10
4/4 [==============================] - 2s 545ms/step - loss: 2.6473 - val_loss: 2.4567
Epoch 9/10
4/4 [==============================] - 2s 483ms/step - loss: 2.3936 - val_loss: 2.3752
Epoch 10/10
4/4 [==============================] - 2s 504ms/step - loss: 2.2760 - val_loss: 2.3759


In [495]:
preds = model.predict_classes(X_test.reshape((X_test.shape[0], X_test.shape[1])))


AttributeError: 'DataFrame' object has no attribute 'reshape'

In [461]:
preds

array([[  1,   5,   9,   0,   0,   0],
       [  8,   7,  11,  10,   0,   0],
       [200,  14,  53, 103,  59,   0],
       [  4,   7, 118,  48,   0,   0],
       [  4,   7,  72,  48,   0,   0]], dtype=int64)

In [462]:
def get_word(n, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == n:
            return word
    return None

In [498]:
preds_text = []
for i in preds:
    temp = []
    for j in range(len(i)):
        t = get_word(i[j], label_tok)
        if j > 0:
            if(t == get_word(i[j-1], label_tok)) or (t == None):
                temp.append('')
            else:
                temp.append(t)
        else:
            if (t == None):
                temp.append('')
            else:
                temp.append(t)
    preds_text.append(' '.join(temp))

In [499]:
preds_text

['1 tbsp cup   ',
 '1 tbsp    ',
 '1 tbsp    ',
 '1 tbsp    ',
 '0 5 cup   ',
 '1 tbsp cup   ',
 '1 tbsp    ',
 '1 tbsp    ',
 '1 tbsp    ',
 'rind   sunflower hemp ',
 '1 tbsp cup   ',
 '1 tbsp cup   ',
 '1 tbsp cup   ',
 '1 tbsp    ',
 '0 5 cup   ',
 '1 tbsp    ',
 '1 5 cup   ',
 '1 tbsp cup   ',
 '1     ',
 '1 tbsp    ',
 '1 tbsp cup   ',
 '0 5 cup   ',
 '1 tbsp cup   ',
 '0 5 cup   ',
 '1 tbsp    ',
 '1 tbsp cup   ',
 '1 tbsp    ',
 '1 tbsp cup   ',
 '0 5 cup   ',
 '1 5 cup   ',
 '1 tbsp cup   ',
 '1 tbsp cup   ',
 '1 tbsp    ',
 '0 5 cup   ',
 '0 5 cup   ',
 '1 tbsp    ',
 '0 5 cup   ',
 '1 tbsp    ',
 '1 tbsp    ',
 '1 tbsp    ',
 '1 tbsp    ',
 '1 tbsp    ',
 '1 tbsp cup   ',
 '1 tbsp    ',
 '0 5 cup   ',
 '1 tbsp cup   ',
 '1 tbsp cup   ',
 '1 tbsp cup   ',
 '0 5 cup   ',
 '1     ',
 '1 tbsp    ',
 '1 tbsp    ',
 '1 tbsp    ',
 '1 tbsp    ',
 '1 tbsp    ',
 '1 tbsp    ',
 '1 tbsp cup   ',
 '1 tbsp    ',
 '1 tbsp    ',
 '1 tbsp cup   ',
 '1 5 cup   ',
 '1 tbsp    ',
 '0 5 cup   

In [465]:
X_train = training_df
Y_train = labels
X_test = pd.read_csv('./data_test.txt', delimiter='\n')

In [466]:
label_tok.word_index

{'1': 1,
 '0': 2,
 'cup': 3,
 '2': 4,
 'tsp': 5,
 '5': 6,
 'tbsp': 7,
 '3': 8,
 'salt': 9,
 'oil': 10,
 'olive': 11,
 'garlic': 12,
 '25': 13,
 'g': 14,
 'oz': 15,
 'lemon': 16,
 'onion': 17,
 '8': 18,
 '4': 19,
 'cloves': 20,
 'seed': 21,
 'lb': 22,
 'water': 23,
 '33': 24,
 'cilantro': 25,
 'chilli': 26,
 'tomato': 27,
 'pepper': 28,
 'clove': 29,
 'almond': 30,
 'egg': 31,
 'can': 32,
 'juice': 33,
 '75': 34,
 'ginger': 35,
 'ground': 36,
 'coconut': 37,
 'parmesan': 38,
 'sesame': 39,
 'green': 40,
 'kale': 41,
 'turmeric': 42,
 'powder': 43,
 'carrot': 44,
 'milk': 45,
 'tofu': 46,
 '6': 47,
 'sauce': 48,
 'chickpea': 49,
 'black': 50,
 'mustard': 51,
 '66': 52,
 'vegetable': 53,
 'flour': 54,
 'cheese': 55,
 'pasta': 56,
 'flake': 57,
 'bean': 58,
 'vinegar': 59,
 'leave': 60,
 'cumin': 61,
 'celery': 62,
 'mushroom': 63,
 'cauliflower': 64,
 'butter': 65,
 '55': 66,
 'cherry': 67,
 'chives': 68,
 'lettuce': 69,
 'pesto': 70,
 'potato': 71,
 'curry': 72,
 'serrano': 73,
 'herb': 

In [454]:
X_test = encode_sequences(training_tok, training_length, X_test['string'])

In [467]:
tf.__version__

'2.5.0'

In [469]:
np.__version__

'1.19.5'

In [470]:
pd.__version__

'1.3.0'

In [497]:
preds = np.argmax(model.predict(X_train), axis=-1)